# Chapter 3: Processing Pipelines (by. huffon)
- 이번 챕터에서는 spaCy 내 **프로세싱 파이프 라인**의 모든 것에 대해 배워볼 것입니다.
- 이번 장을 통해 nlp 객체를 호출해 텍스트를 처리할 때 spaCy 내에서 어떤 작업이 발생하는지 학습하게 될 것이고,
- 어떻게 **자신만의 컴포넌트**를 작성하고, 이를 기존 파이프 라인에 추가할 수 있을지에 대해 배워볼 것입니다.
- 또한 프로젝트에 용이하게 사용될 수 있는 자신만의 메타 데이터를 Doc, Span 그리고 Token 객체에 추가하는 방법 역시 학습합니다.
<br/><br/>


## 1. Processing pipelines

<img src="https://course.spacy.io/pipeline.png" width=500></img>

- 파이프 라인 기본 내장 컴포넌트
    1. `tagger`: Part-of-speech tagger
    2. `parser`: Dependency parser
    3. `ner`: Named entity recognizer
    4. `textcat`: Text classifier

<br/><br/>
   
<img src="https://course.spacy.io/package_meta.png" width=500></img>

- 모델마다의 파이프 라인을 구성하는 요소가 다르며, 모델 내 `meta.json` 파일에 파이프 라인이 정의되어 있음

### Inspecting the pipeline

- `en_core_web_sm` 모델을 로드하고 nlp 객체를 생성하세요
- `nlp.pipe_names`을 입력해 파이프 라인을 구성하는 컴포넌트들의 명칭을 출력하세요
- `nlp.pipeline`을 입력해 (명칭, 객체)로 구성된 튜플들을 출력하세요

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

print(nlp.pipe_names, '\n')

print(nlp.pipeline)

## 2. Custom pipeline components
- 왜 커스텀 컴포넌트가 필요할까요?
    - `nlp` 객체를 생성할 때, 특정 함수를 자동으로 적용하기 위해
    -  documents 객체와 token 객체들에 특정 메타 데이터를 추가해주기 위해
  
<br/>
    
- 커스텀 컴포넌트 옵션
    1. first: 파이프 라인의 첫 번째 자리에 커스텀 컴포넌트 추가
        - nlp.add_pipe(component, first=True)
    2. last: 파이프 라인의 마지막 자리에 커스텀 컴포넌트 추가
        - nlp.add_pipe(component, last=True)
    3. before: 특정 컴포넌트 앞에 커스텀 컴포넌트 추가
        - nlp.add_pipe(component, before='ner')
    4. after: 특정 컴포넌트 뒤에 커스텀 컴포넌트 추가
        - nlp.add_pipe(component, after='tagger')

### Simple components
- doc 객체의 길이를 구하는 컴포넌트 함수 `length_component`를 완성하세요
- 앞서 작성한 `length_component` 함수를 존재하는 파이프 라인의 첫 번째 컴포넌트로 추가하세요
- 새로운 컴포넌트가 추가된 파이프 라인을 이용해 nlp 객체의 텍스트를 분석해보세요

In [ ]:
def length_component(doc):
    doc_length = len(doc)
    print("This document is {} tokens long.".format(doc_length))
    return doc

nlp = spacy.load("en_core_web_sm")

nlp.add_pipe(length_component, first=True)
print(nlp.pipe_names)

doc = nlp("This is a sentence.")

### Complex components
- matcher를 doc 객체에 적용하기 위한 커스텀 컴포넌트를 정의하세요
- 매칭 결과에 대한 Span 객체를 생성한 후, 해당 객체에 'ANIMAL' 라벨을 부여한 후 doc 객체의 entity로 추가하세요
- 앞서 정의한 커스텀 컴포넌트를 'ner' 컴포넌트 뒤에 위치하도록 추가하세요
- 입력 문장을 nlp 객체에 전달한 후, 파이프 라인을 거쳐 생성된 엔티티를 출력해보세요

In [ ]:
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", None, *animal_patterns)

def animal_component(doc):
    matches = matcher(doc)
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    doc.ents = spans
    return doc


nlp.add_pipe(animal_component, after="ner")
print(nlp.pipe_names)

doc = nlp("I have a cat and a Golden Retriever")
print([(ent.text, ent.label_) for ent in doc.ents])

## 3. Extension attrtibutes
- document, token, span 객체에 커스텀 메타 데이터를 추가해줄 수 있습니다
- 객체의 속성에 접근하기위해서는 `._` 프로퍼티를 사용해야 합니다

```
doc._.title = 'My document'
token._.is_color = True
span._.has_color = False
```

- `set_extension` 메소드를 이용해 글로벌 Doc, Token, Span 객체에 추가도 가능합니다

```
from spacy.tokens import Doc, Token, Span

Doc.set_extension('title', default=None)
Token.set_extension('is_color', default=False)
Span.set_extension('has_color', default=False)
```

- getter 함수를 추가로 정의해줄 수도 있습니다
- getter는 사용자가 속성 값을 확인하고자 할 때 호출됩니다

```
from spacy.tokens import Token

def get_is_color(token):
    colors = ['red', 'yellow', 'blue']
    return token.text in colors

Token.set_extension('is_color', getter=get_is_color)

doc = nlp("The sky is blue.")
print(doc[3]._.is_color, '-', doc[3].text)
```

- 속성의 추가 뿐만 아니라 사용자 정의 함수를 추가해줄 수도 있습니다

```
from spacy.tokens import Doc

def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

Doc.set_extension('has_token', method=has_token)

doc = nlp("The sky is blue.")
print(doc._.has_token('blue'), '- blue')
print(doc._.has_token('cloud'), '- cloud')
```

### Setting extension attributes (1)
- default 값이 `False`인 `is_country` 속성을 Token.set_extension을 이용해 등록하세요
- 해당 속성을 `"Spain"`으로 업데이트 한 후, 모든 토큰들을 출력하세요

In [ ]:
from spacy.lang.en import English
from spacy.tokens import Token

nlp = English()
# nlp = spacy.load("en_core_web_sm")

Token.set_extension("is_country", default=False)

doc = nlp("I live in Spain.")
print(doc.ents)
doc[3]._.is_country = True

print([(token.text, token._.is_country) for token in doc])

### Setting extension attributes (2)
- `has_number` 함수를 완성하세요
- `Doc.set_extension` 메소드를 이용해 has_number 함수를 등록하세요

In [ ]:
from spacy.lang.en import English
from spacy.tokens import Doc

nlp = English()

def get_has_number(doc):
    return any(token.like_num for token in doc)

Doc.set_extension("has_number", getter=get_has_number)

doc = nlp("The museum closed for five years in 2012.")
print("has_number:", doc._.has_number)

### Entities and extensions
- Span 객체의 라벨이 ['사람', '지역', '기관', '장소']에 속하면 위키피디아 주소를 반환해주는 get_wikipedia_url 함수를 완성하세요
- Span.set_extension 메소드를 이용해 앞서 정의한 'wikipedia_url' 함수를 추가하세요
- doc 객체 내 엔티티들을 순회하며, 각 엔티티의 위키피디아 주소를 출력하세요

In [ ]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")

def get_wikipedia_url(span):
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text


Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "In over fifty years from his very first recordings right through to his "
    "last album, David Bowie was at the vanguard of contemporary culture."
)

for ent in doc.ents:
    print(ent.text, ent._.wikipedia_url)